In [1]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm import tqdm
import pandas as pd
import pymongo
import pickle
import json
from copy import deepcopy
from pymongo import MongoClient

In [34]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='sw-dbpedia.log', mode='a')
formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s",
                              "%Y-%m-%d %H:%M:%S")
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)


In [2]:
GRAPH_1_INV_FUNC_PATH = ''
GRAPH_1 = ""
GRAPH_2_INV_FUNC_PATH = ''
GRAPH_2 = "http://localhost:9999/bigdata/sparql"
SUBJECT = 'subject'
PREDICATE = 'predicate'
INVERSE_FUNCTIONALITY = 'inverse_functionality'


In [3]:
graph_1_inv_func_df = pd.read_csv(GRAPH_1_INV_FUNC_PATH)

threshold_graph_1_inv_func_df = graph_1_inv_func_df[graph_1_inv_func_df[INVERSE_FUNCTIONALITY] > 0.5]

graph_1_predicate_list = threshold_graph_1_inv_func_df[PREDICATE].tolist()

graph_1 = Graph()
graph_1.parse(GRAPH_1)

http://www.playstationlifestyle.net/2010/11/04/ps3-review-star-wars-the-force-unleashed-ii%7Cdate=2010-10-29%7Caccessdate=2013-05-26%7Cpublisher=playstationlife\nstyle.net does not look like a valid URI, trying to serialize this will break.
http://www.playstationlifestyle.net/2010/11/04/ps3-review-star-wars-the-force-unleashed-ii%7Cdate=2010-10-29%7Caccessdate=2013-05-26%7Cpublisher=playstationlife\nstyle.net does not look like a valid URI, trying to serialize this will break.
http://www.playstationlifestyle.net/2010/11/04/ps3-review-star-wars-the-force-unleashed-ii%7Cdate=2010-10-29%7Caccessdate=2013-05-26%7Cpublisher=playstationlife\nstyle.net does not look like a valid URI, trying to serialize this will break.
http://www.playstationlifestyle.net/2010/11/04/ps3-review-star-wars-the-force-unleashed-ii%7Cdate=2010-10-29%7Caccessdate=2013-05-26%7Cpublisher=playstationlife\nstyle.net does not look like a valid URI, trying to serialize this will break.
http://www.playstationlifestyle.net/

<Graph identifier=N5a58628b729c4aabb9a6b7b1a5324efb (<class 'rdflib.graph.Graph'>)>

In [4]:
literal_y1_dict = dict()

for index, row in tqdm(threshold_graph_1_inv_func_df.iterrows(), total=len(threshold_graph_1_inv_func_df)):
    predicate = row[PREDICATE]
    predicate_uri = URIRef(predicate)
    
    for s, o in graph_1.subject_objects(predicate_uri):
        if type(o) == Literal and o.value and type(o.value) == str:
            o_value = o.value
            if o_value not in literal_y1_dict.keys():
                literal_y1_dict[o_value] = list()
            y1_value_list = literal_y1_dict[o_value]
            y1_value_list.append({
                SUBJECT: s,
                PREDICATE: predicate_uri
            })

100%|█████████████████████████████████████████| 640/640 [00:07<00:00, 84.20it/s]


In [5]:
graph_2_inv_func_df = pd.read_csv(GRAPH_2_INV_FUNC_PATH)

threshold_graph_2_inv_func_df = graph_2_inv_func_df[graph_2_inv_func_df[INVERSE_FUNCTIONALITY] > 0.5]

graph_2_predicate_list = threshold_graph_2_inv_func_df[PREDICATE].tolist()

#graph_2 = Graph()
#graph_2.parse(GRAPH_2)
graph_2 = Graph("SPARQLStore")
graph_2.open(GRAPH_2)

In [6]:
y2_matches = dict()

for index, row in tqdm(threshold_graph_2_inv_func_df.iterrows(), total=len(threshold_graph_2_inv_func_df)):
    predicate = row[PREDICATE]
    predicate_uri = URIRef(predicate)
    
    for s, o in graph_2.subject_objects(predicate_uri):
        if type(o) != Literal or not o.value or type(o.value) != str:
            continue
        
        # Check if exist match
        o_value = o.value
        if o_value not in literal_y1_dict.keys():
            continue
            
        if o_value not in y2_matches.keys():
            y2_matches[o_value] = list()
        y2_match_list = y2_matches[o_value]
        y2_match_list.append({
            SUBJECT: s,
            PREDICATE: predicate_uri
        })

  1%|▍                                      | 419/37986 [11:43<38:09, 16.41it/s]Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-1005-10-06'
  1%|▍                                      | 444/37986 [11:45<58:39, 10.67it/s]Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call la

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-1949-10-31'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-1948-11-30'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0507-09-01'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0401-09-03'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0207-06-23'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0049-02-21'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0042-10-23'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x110e31310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv

http://dbpedia.org/class/yago/WikicatAlbumsProducedByJohnny"J" does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Albums_produced_by_Johnny_"J" does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatAlbumsProducedByRobertJohn"Mutt"Lange does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Albums_produced_by_Robert_John_"Mutt"_Lange does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatAlbumsProducedByRon"Pigpen"McKernan does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Albums_produced_by_Ron_"Pigpen"_McKernan does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatAll-UkrainianUnion"Fatherland"Politicians does not look like a valid URI, trying to serialize 

http://dbpedia.org/class/yago/WikicatMoldovan"A"DivisionSeasons does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Moldovan_"A"_Division_seasons does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatMoldovan"B"DivisionSeasons does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Moldovan_"B"_Division_seasons does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatMusicVideosDirectedBy"WeirdAl"Yankovic does not look like a valid URI, trying to serialize this will break.
http://yago-knowledge.org/resource/wikicat_Music_videos_directed_by_"Weird_Al"_Yankovic does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatNationalConvergence"KwaNaKwa"Politicians does not look like a valid URI, trying to serialize this will break.
h

In [7]:
with open('y2_matches_starwars_dbpedia.pkl', 'wb') as f:
    pickle.dump(y2_matches, f, pickle.HIGHEST_PROTOCOL)

In [8]:
client = MongoClient()
db = client['PARIS-db']
entity_pair_match_collection = db['entity_pair_match']
entity_pair_match_collection.drop()
entity_pair_match_collection.create_index([('s1', pymongo.ASCENDING), ('s2', pymongo.ASCENDING)])
predicate_pair_collection = db['predicate_pair_collection']
predicate_pair_collection.drop()
predicate_pair_collection.create_index([('p1', pymongo.ASCENDING), ('p2', pymongo.ASCENDING)])

'p1_1_p2_1'

In [9]:
def get_predicate_prob(predicate_pair_dict, p1, p2):
    if p1 not in predicate_pair_dict.keys():
        predicate_pair_dict[p1] = dict()
    if p2 not in predicate_pair_dict[p1].keys():
        predicate_pair_dict[p1][p2] = 0.1

    return predicate_pair_dict[p1][p2]

In [10]:
inv_func_1_dict = dict()

for index, row in tqdm(threshold_graph_1_inv_func_df.iterrows(), total=len(threshold_graph_1_inv_func_df)):
    inv_func_1_dict[row['predicate']] = row['inverse_functionality']

100%|██████████████████████████████████████| 640/640 [00:00<00:00, 19820.24it/s]


In [11]:
inv_func_2_dict = dict()

for index, row in tqdm(threshold_graph_2_inv_func_df.iterrows(), total=len(threshold_graph_2_inv_func_df)):
    inv_func_2_dict[row['predicate']] = row['inverse_functionality']

100%|██████████████████████████████████| 37986/37986 [00:00<00:00, 61386.28it/s]


In [12]:
def save_predicate_pair(p1, p2, match):
    predicate_pair = predicate_pair_collection.find_one({'p1': str(p1),
                                                       'p2': str(p2)})
    if predicate_pair is None:
        predicate_pair_collection.insert_one({
            'p1': str(p1),
            'p2': str(p2),
            'matches': [match]
        })
    else:
        predicate_pair['matches'].append(match)

        predicate_pair_collection.update_one({"_id": predicate_pair['_id']},{"$set": {"matches": predicate_pair['matches']}})


In [13]:
def update_prob_entities(entity_pair_match_dict, x1, x2, p1, p2, prob_y):
    if x1 not in entity_pair_match_dict.keys():
        entity_pair_match_dict[x1] = dict()
    if x2 not in entity_pair_match_dict[x1].keys():
        entity_pair_match_dict[x1][x2] = 1
    
    y1_pred = str(p1)
    y2_pred = str(p2)
    pred_prob_1_2 = get_predicate_prob(predicate_pair_dict, y1_pred, y2_pred)
    pred_prob_2_1 = get_predicate_prob(predicate_pair_dict, y2_pred, y1_pred)

    new_factor = (1-pred_prob_1_2*inv_func_1_dict[y1_pred] * prob_y) * (1-pred_prob_2_1*inv_func_2_dict[y2_pred] * prob_y)

    entity_pair_match_dict[x1][x2] *= new_factor

In [37]:
def calculate_entity_prob_using_attribute(new_entity_pair_match_dict):
    logger.info("Total attributes: " + str(len(y2_matches.items())))
    counter = 0
    for o_value, y2_match_list in y2_matches.items():
        counter += 1
        if counter % 100 == 0:
            logger.info("Calculated attributes: " + str(counter))
        y1_match_list = literal_y1_dict[o_value]

        for y1_match in y1_match_list:
            y1_subject = y1_match['subject']

            for y2_match in y2_match_list:
                y2_subject = y2_match['subject']


                y1_pred = str(y1_match['predicate'])
                y2_pred = str(y2_match['predicate'])
                update_prob_entities(new_entity_pair_match_dict, str(y1_subject), str(y2_subject), y1_pred, y2_pred, 1)

                match = {'e1': str(y1_subject), 'e2': str(y2_subject), 'sim': 1}
                save_predicate_pair(y1_pred, y2_pred, match)
                save_predicate_pair(y2_pred, y1_pred, match)

In [15]:
def get_inbound_neighbors(graph, predicate_list, y):
    if not y:
        return []
    y_url = y
    if type(y_url) != URIRef:
        y_url = URIRef(y)
    
    neighbor_list = list()
    for s, p in graph.subject_predicates(y_url):
        if str(p) in predicate_list:
            neighbor_list.append({
                'p': p,
                's': s
            })
    return neighbor_list

In [17]:
def update_prob_entity_neighbors(new_entity_pair_match_dict, old_entity_pair_match_dict, y1_neighbors, y2_neighbors, y_sim):
    for x1 in y1_neighbors:
        for x2 in y2_neighbors:
            update_prob_entities(new_entity_pair_match_dict, str(x1['s']), str(x2['s']), str(x1['p']), str(x2['p']), \
                                 y_sim)
            match = {'e1': str(x1), 'e2': str(x2), 'sim': y_sim}
            save_predicate_pair(x1['p'], x2['p'], match)
            save_predicate_pair(x2['p'], x1['p'], match)

In [36]:
def calculate_entity_prob_using_neighbors(new_entity_pair_match_dict, old_entity_pair_match_dict):
    logger.info("Total x1 entities: " + str(len(old_entity_pair_match_dict.keys())))
    counter = 0
    for y1 in old_entity_pair_match_dict.keys():
        logger.info("Analyzing entities neighbors: " + str(y1))
        if counter % 100 == 0:
            logger.info("Calculated neighbors of " + str(counter))
        counter += 1
        logger.info("Analyzed " + str(counter))
        y1_neighbors = get_inbound_neighbors(graph_1, graph_1_predicate_list, y1)
        for y2 in old_entity_pair_match_dict[y1]:
            y2_neighbors = get_inbound_neighbors(graph_2, graph_2_predicate_list, y2)
            y_sim = 1 - old_entity_pair_match_dict[y1][y2]
            update_prob_entity_neighbors(new_entity_pair_match_dict, old_entity_pair_match_dict, y1_neighbors, y2_neighbors, y_sim)
    

In [20]:
def get_prob_entity_pair(entity_pair_match_dict, s1, s2):
    if s1 in entity_pair_match_dict.keys() and s20 in entity_pair_match_dict[s1].keys():
        return 1 - entity_pair_match_dict[s1][s2]
    return 0

In [21]:
def calculate_prob_predicates(predicate_pair_dict, entity_pair_match_dict):
    for p1 in predicate_pair_dict.keys():
        lower_sum = 0
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair = predicate_pair_collection.find_one({'p1': str(p1),
                                                       'p2': str(p2)})
            
            if predicate_pair is None:
                continue
            predicate_pair_list = predicate_pair['matches']

            sum_prob_entities = 0
            for match in predicate_pair_list:
                prob_entity = (1-entity_pair_match_dict[match['e1']][match['e2']])
                prob_obj = match['sim']
                sum_prob_entities += prob_entity * prob_obj
            predicate_pair_dict[p1][p2] = sum_prob_entities
            lower_sum += predicate_pair_dict[p1][p2]

        if lower_sum == 0:
            continue
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair_dict[p1][p2] = predicate_pair_dict[p1][p2] / lower_sum

In [22]:
def get_subject_object(graph, predicate):
    s_o_list = list()
    
    for s, o in graph.subject_objects(URIRef(predicate)):
        s_o_list.append({
                'subject': s,
                'object': o
            })
    
    return s_o_list

In [23]:
def check_if_string(y):
    return type(y) == Literal and y.value and type(y.value) == str

In [24]:
def calculate_sub_pred_product(entity_pair_match_dict, x1, x2, y1, y2):
    if check_if_string(y1) and check_if_string(y2):
        if y1.value != y2.value:
            return 0.0
        y_value = y1.value
        if y_value not in literal_y1_dict.keys() or y_value not in y2_matches.keys():
            return 0.0
        return entity_pair_match_dict[str(x1)][str(x2)]
    #TODO: Write the case when y is entity
    #else if type(y1) == URIRef:
    #    y1_value = str(y1)
    #    y2_value = str(y2)
    #    if y1_value not in entity_pair_match_dict.keys() or y2_value not in entity_pair_match_dict[y1_value].keys():
    #        return 0.0
    #    y1_neighbors = get_inbound_neighbors(graph_1, graph_1_predicate_list, y1_value)
    #    y2_neighbors = get_inboud_neighbors(graph2, graph_2_predicate_list, y2_value)
    return 0.0

def calculate_prob_predicates(predicate_pair_dict, entity_pair_match_dict):
    for p1 in tqdm(predicate_pair_dict.keys()):
        s_o_1_list = get_subject_object(graph_1, p1)
        lower_sum = 0
        
        filtered_s_o_1_list = list()
        for so1 in s_o_1_list:
            if check_if_string(so1['object']) and so1['object'].value in y2_matches.keys():
                filtered_s_o_1_list.append(so1)
            #TODO implement for entities
        
        for p2 in tqdm(predicate_pair_dict[p1]):
            s_o_2_list = get_subject_object(graph_2, p2)
            
            filtered_s_o_2_list = list()
            for so2 in s_o_2_list:
                if check_if_string(so2['object']) and so2['object'].value in y2_matches.keys():
                    filtered_s_o_2_list.append(so2)
            #TODO implement for entities
            
            upper_sum = 0
            for s_o_1 in filtered_s_o_1_list:
                mul_fact = 1
                
                for s_o_2 in filtered_s_o_2_list:
                    mul_fact *= 1 - calculate_sub_pred_product(entity_pair_match_dict, s_o_1['subject'], s_o_2['subject'], s_o_1['object'], s_o_2['object'])
                    
                upper_sum += 1 - mul_fact
            
            predicate_pair_dict[p1][p2] = upper_sum
            lower_sum += upper_sum
        
        for p2 in predicate_pair_dict[p1]:
            predicate_pair_dict[p1][p2] = predicate_pair_dict[p1][p2] / lower_sum
            
            

In [25]:
def save_entity_pairs(i, entity_pair_match_dict):
    entity_pair_sim_list = list()
    for y1 in entity_pair_match_dict.keys():
        max_sim = 0
        max_entity = ""
        for y2 in entity_pair_match_dict[y1].keys():
            similarity = 1 - entity_pair_match_dict[y1][y2]
            if similarity > max_sim:
                max_sim = similarity
                max_entity = y2
        entity_pair_sim_list.append({
            "e1": y1,
            "e2": y2,
            "sim": similarity
        })
    entity_sim_df = pd.DataFrame(entity_pair_sim_list)
    entity_sim_df.to_csv("entity_sim_ff_nintendo_" + str(i) + ".csv")

In [26]:
def save_predicate_pairs_into_df(i, predicate_pair_dict):
    predicate_pair_list = list()
    for p1 in predicate_pair_dict.keys():
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair_list.append({
                "p1": p1,
                "p2": p2,
                "sim": predicate_pair_dict[p1][p2]
            })
    predicate_pair_df = pd.DataFrame(predicate_pair_list)
    predicate_pair_df.to_csv("predicate_pair_ff_nintendo_" + str(i) + ".csv")

In [38]:
predicate_pair_dict = dict()
old_entity_pair_match_dict = dict()

for i in tqdm(range(10)):
    predicate_pair_collection.drop()
    #new_entity_pair_match_dict = dict()
    
    logger.info("Calculate entities similarity using attributes")
    print("Calculate entities similarity using attributes")
    calculate_entity_prob_using_attribute(old_entity_pair_match_dict)
    print("End calculating entities similarity using neighbors")
    logger.info("End calculating entities similarity using neighbors")
    #if i == 0:
    new_entity_pair_match_dict = deepcopy(old_entity_pair_match_dict)
    
    print("Calculate entities similarity using neighbors")
    logger.info("Calculate entities similarity using neighbors")
    calculate_entity_prob_using_neighbors(new_entity_pair_match_dict, old_entity_pair_match_dict)
    
    logger.info("Calculate sub predicate")
    #calculate_prob_predicates(predicate_pair_dict, new_entity_pair_match_dict)
    calculate_prob_predicates(predicate_pair_dict, new_entity_pair_match_dict)
    
    save_entity_pairs(i, new_entity_pair_match_dict)
    save_predicate_pairs_into_df(i, predicate_pair_dict)
    
    old_entity_pair_match_dict = new_entity_pair_match_dict


  0%|                                                    | 0/10 [00:00<?, ?it/s]

Calculate entities similarity using attributes


  0%|                                                 | 0/10 [49:44:45<?, ?it/s]


WriteError: Resulting document after update is larger than 16777216, full error: {'index': 0, 'code': 17419, 'errmsg': 'Resulting document after update is larger than 16777216'}

In [75]:
predicate_pair_dict

In [37]:
entity_sim_df = pd.DataFrame(entity_pair_sim_list)
entity_sim_df.to_csv("entity_sim_ff_nintendo_10.csv")

In [26]:
entity_pair_sim_list = list()
for y1 in new_entity_pair_match_dict.keys():
    max_sim = 0
    max_entity = ""
    for y2 in new_entity_pair_match_dict[y1].keys():
        similarity = 1 - new_entity_pair_match_dict[y1][y2]
        if similarity > max_sim:
            max_sim = similarity
            max_entity = y2
    entity_pair_sim_list.append({
        "e1": y1,
        "e2": y2,
        "sim": similarity
    })
entity_sim_df = pd.DataFrame(entity_pair_sim_list)

In [27]:
pd.set_option('display.max_colwidth', 100)

In [30]:
threshold_entity_sim_df = entity_sim_df[entity_sim_df['sim'] ==  1.0]

In [31]:
threshold_entity_sim_df.to_csv("entity_sim_ff_nintendo_10-1.csv")

In [67]:
predicate_pair_list = list()
for p1 in predicate_pair_dict.keys():
    for p2 in predicate_pair_dict[p1].keys():
        predicate_pair_list.append({
            "p1": p1,
            "p2": p2,
            "sim": predicate_pair_dict[p1][p2]
        })
predicate_pair_df = pd.DataFrame(predicate_pair_list)

In [69]:
predicate_pair_df[predicate_pair_df['sim'] > 0.1]

,p1,p2,sim
1305,http://dbkwik.webdatacommons.org/finalfantasy./property/first,http://dbkwik.webdatacommons.org/nintendo./ontology/wikiPageWikiLinkText,0.11753
1306,http://dbkwik.webdatacommons.org/finalfantasy./property/first,http://www.w3.org/2000/01/rdf-schema#label,0.11753
1307,http://dbkwik.webdatacommons.org/finalfantasy./property/first,http://dbkwik.webdatacommons.org/nintendo./property/name,0.11753
1308,http://dbkwik.webdatacommons.org/finalfantasy./property/first,http://www.w3.org/2004/02/skos/core#altLabel,0.11749
1309,http://dbkwik.webdatacommons.org/finalfantasy./property/first,http://www.w3.org/2004/02/skos/core#prefLabel,0.11753
...,...,...,...
17022,http://dbkwik.webdatacommons.org/finalfantasy./property/4Parry,http://dbkwik.webdatacommons.org/nintendo./property/boss,0.50000
17023,http://dbkwik.webdatacommons.org/finalfantasy./property/4Parry,http://dbkwik.webdatacommons.org/nintendo./property/enemy,0.50000
17024,http://dbkwik.webdatacommons.org/nintendo./property/enemy,http://dbkwik.webdatacommons.org/finalfantasy./property/3Lift,0.50000
17025,http://dbkwik.webdatacommons.org/nintendo./property/enemy,http://dbkwik.webdatacommons.org/finalfantasy./property/4Parry,0.50000


In [71]:
predicate_pair_dict['http://dbkwik.webdatacommons.org/nintendo./ontology/wikiPageWikiLinkText']

{'http://dbkwik.webdatacommons.org/finalfantasy./ontology/wikiPageWikiLinkText': 0.0020778065635624738,
 'http://www.w3.org/2000/01/rdf-schema#label': 0.0020778065635624738,
 'http://www.w3.org/2004/02/skos/core#prefLabel': 0.0020778065635624738,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/title': 0.0020778065635624738,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/first': 0.0020778065635624738,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/firstrelease': 0.0020778065635624694,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/loves': 0.0020777982393128443,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/asText': 0.002012617805058709,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/name': 0.0020778065635624738,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/qsText': 0.0020776406108025903,
 'http://dbkwik.webdatacommons.org/finalfantasy./property/3d': 0.0020778065635624738,
 'http://dbkwik.webdatacommons.org/fi